In [1]:
import os
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
import json
from decouple import AutoConfig
from langdetect import detect
import time
import sqlite3
import bs4 as bs

In [2]:
config = AutoConfig(search_path=os.getcwd())

In [3]:
def delta_date(start_date,end_date):
    """Function that returns the number of days between 2 dates """

    return abs((datetime.strptime(start_date, "%Y-%m-%d") - datetime.strptime(end_date, "%Y-%m-%d")).days)

In [4]:
def get_tickers():
    """Method that gets the stock symbols from companies listed in the S&P 500
    Return
    ------
    `tickers` : list
        S&P 500 company symbols
    """
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find_all('table')[0]  # Grab the first table

    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip('\n')
        tickers.append(ticker)

    return tickers

In [5]:
class Init():
    """Class that initializes global value for the module. It also use general method to initialize value."""

    def __init__(self):
        """Built-in method to inialize the global values for the module
        Attributes
        -----------
        `self.start.date` : str
            start date of the training period. Must be within the last year for the free version of FinHub. Format
            must be "YYYY-mm-dd"
        `self.end_date` : str
            end date of the training period. Format must be "YYYY-mm-dd"
        `self.ticker` : list
            tickers on which we want to perform the test. Can be one ticker in form of a list as well as a list
            of tickers like the s&p 500.
        `self.db_name` : str
            name of the sqlite3 database
        `self.dir_path` : str
            directory where the data are saved. It takes into account the `self.start_date` and `self.end_date`
        `self.start_date_` : datetime object
            same thing as `start_date` but as a datetime object
        `self.end_date_` : datetime object
            same thing as `start_date` but as a datetime object
        """

        #initialize value here
        self.start_date = "2022-09-29"
        self.end_date = "2023-05-29"
        self.tickers = ['AAPL']

        self.db_name = 'financial_data'
        self.dir_path = os.path.realpath('') + '/output/' + self.start_date + '_' + self.end_date + '/'
        Path(self.dir_path).mkdir(parents=True, exist_ok=True) #create new path if it doesn't exist
        self.start_date_ = datetime.strptime(self.start_date, "%Y-%m-%d")  #datetime object
        self.end_date_ = datetime.strptime(self.end_date, "%Y-%m-%d")    #datetime object
        self.delta_date = abs((self.end_date_ - self.start_date_).days) #number of days between 2 dates


        try:
            self.start_date_ > self.end_date_
        except:
            print("'start_date' is after 'end_date'")

        t = (datetime.now()- relativedelta(years=1))
        d= datetime.strptime(self.start_date, "%Y-%m-%d")

        if (datetime.strptime(self.start_date, "%Y-%m-%d") <= (datetime.now()- relativedelta(years=1))) :
            raise Exception("'start_date' is older than 1 year. It doesn't work with the free version of FinHub")

In [6]:
class FinnHub():
    """Class to make API calls to FinnHub"""

    def __init__(self,start_date,end_date,start_date_,end_date_,tickers,dir_path,db_name):
        """ Class constructor
        Parameters
        ----------
        `start_date` : str
            Start date of the request. Must be within 1 year from now for must request
            with the free version of FinHub
        `end_date` : str
            End date of the request.
        `start_date_` : datetime object
            Same thing as `start_date` but as a datetime object
        `end_date_` : datetime object
             Same thing as `start_date` but as a datetime object
        `ticker` : str
            Ticker symbol
        `db_name` : str
            Name of the sqlite database
        `dir_path` : str
            Directory  where our data will be stored
        Attributes
        ----------
        `self.max_call` : int
            maximum api calls per minute for the finhub API
        `self.time_sleep` : int
            seconds to sleep before making a new API call. Default is 60 seconds as the maximum number of API calls is
            per minute
        `self.nb_request` : int
            nb of request made so far. Set to 0 in constructor `__init__` as we may loop through ticker
            and want to avoid the variable to reset to 0 when exiting the wrapper `iterate_day()` (which could generate
            an error)
        `self.finhub_key` : str
            finhub unique API key. Get yours here : https://finnhub.io/
        `self.db_name : str
            default file name for the sql database
        """

        #Initialize attributes values here
        self.max_call = 60
        self.time_sleep = 60
        self.nb_request = 0
        self.finhub_key = config('FINHUB_KEY')
        self.news_header = ['category', 'datetime','headline','id','image','related','source','summary','url']
        self.start_date = start_date
        self.end_date = end_date
        self.tickers = tickers
        self.ticker_request = tickers #different value because ticker like 'ALL' (All State) can generate error in SQLite
                                    #database
        self.dir_path = dir_path
        self.db_name = db_name
        self.js_data = []

        self.start_date_ = start_date_ #datetime object
        self.end_date_ = end_date_ #datetime object

        #call the methods to access historical financial headlines
        #tickers = get_tickers() #get_tickers is to get tickers from all the companies listedin the s&p 500

        for ticker_ in self.tickers:
            self.js_data.clear()
            self.ticker = ticker_ + '_'
            self.ticker_request = ticker_
            self.req_new()
            self.create_table()
            self.clean_table()
            self.lang_review()

    def init_sql(func):
        """ Decorator that open the sql database, save it and close it. The operation are between the opening and
        saving of the file"""

        def wrapper_(self):
            conn_ = sqlite3.connect(self.dir_path + self.db_name + '.db')
            c = conn_.cursor()
            func(self,conn_,c)
            conn_.commit()
            conn_.close()
        return wrapper_

    @init_sql
    def clean_table(self,conn_,c):
        """Method that clean the database using sqlite3
        Parameters
        ----------
        `conn_` : database object
            Connection object that represents the database
        `c` : database object
            Cursor object
        """

        #remove NULL entry (row) from headline column
        c.execute(f" DELETE FROM {self.ticker} WHERE {self.news_header[2]} IS NULL OR "
                  f"trim({self.news_header[2]}) = '';")
        # remove NULL value from datetime
        c.execute(f" DELETE FROM {self.ticker} WHERE {self.news_header[1]} IS NULL OR "
                  f"trim({self.news_header[1]}) = '';")

        #removes duplicate entries (row)
        c.execute(f" DELETE FROM {self.ticker} WHERE rowid NOT IN (select MIN(rowid)"
                  f"FROM {self.ticker} GROUP BY {self.news_header[2]})")

    @init_sql
    def create_table(self,conn_,c):
        """ Method that creates a table in SQLite database. It creates the table  in `self.dir_path` and write
        the data in it
        Parameters
        ----------
        `conn_` : database object
            Connection object that represents the database
        `c` : database object
            Cursor object
        """

        #create table if it does not exist
        c.execute(f'drop table if exists {self.ticker}')
        conn_.commit()
        c.execute(f"CREATE TABLE IF NOT EXISTS {self.ticker} ({self.news_header[0]})")
        conn_.commit()

        #add columns to the table if the columns don't exist
        for header_ in range(len(self.news_header)-1):
            c.execute(f"alter table {self.ticker} add column '%s' " % self.news_header[header_+1])
            conn_.commit()

        iteration = 0
        for data_ in self.js_data:
            iteration +=1
            try :
                c.execute(f'insert into {self.ticker} values (?,?,?,?,?,?,?,?,?)',[data_[self.news_header[0]],
                          data_[self.news_header[1]],data_[self.news_header[2]],data_[self.news_header[3]],
                        data_[self.news_header[4]],data_[self.news_header[5]],data_[self.news_header[6]],
                          data_[self.news_header[7]],data_[self.news_header[8]]])
            except:
                print(f"Error at the {iteration}th ieration")

            conn_.commit()

    def iterate_day(func):
        """ Decorator that makes the API call on FinHub each days between the `self.start_date`
        and `self.end_date` """

        def wrapper_(self):
            delta_date_ = delta_date(self.start_date,self.end_date)
            date_ = self.start_date
            date_obj = self.start_date_

            for item in range(delta_date_ + 1):
                self.nb_request +=1
                func(self,date_)
                date_obj = date_obj + relativedelta(days=1)
                date_  = date_obj.strftime("%Y-%m-%d")
                if self.nb_request == (self.max_call-1):
                    time.sleep(self.time_sleep)
                    self.nb_request=0
        return wrapper_

    @init_sql
    def lang_review(self,conn_,c):
        """ Methods that delete non-english entries based on the 'headline' column in a SQLlite3 db
        Parameters
        ----------
        `conn_` : database object
            Connection object that represents the database
        `c` : database object
            Cursor object
        """

        list_ = []
        c.execute(f" SELECT {self.news_header[2]} FROM {self.ticker}")

        #check for non-english headlines
        for item_ in c:
            if detect(item_[0]) != 'en':
                list_.append(item_[0])

        #delete non-english entries (rows)
            query = f"DELETE FROM {self.ticker} where {self.news_header[2]} in ({','.join(['?']*len(list_))})"
            c.execute(query, list_)

    @iterate_day
    def req_new(self,date_):
        """ Method that makes news request(s) to the Finnhub API"""

        request_ = requests.get('https://finnhub.io/api/v1/company-news?symbol=' + self.ticker_request + '&from=' +
                                date_ + '&to=' + date_ + '&token=' + self.finhub_key)
        self.js_data += request_.json()

In [7]:
init_ = Init()

In [8]:
init_.tickers

['AAPL']

In [9]:
%%time

finhub = FinnHub(start_date=init_.start_date, end_date=init_.end_date,start_date_=init_.start_date_ ,
                end_date_ =init_.end_date_, tickers=init_.tickers, dir_path =init_.dir_path,db_name=init_.db_name)

CPU times: user 5.98 s, sys: 5.17 s, total: 11.2 s
Wall time: 5min 43s


## Попробуем обратиться к базе данных

In [10]:
conn = sqlite3.connect(r'/Users/macbook/PycharmProjects/pipenv_demo/output/2022-09-29_2023-05-29/financial_data.db')

In [11]:
cur = conn.cursor()

In [12]:
cur.execute("SELECT count(*) FROM aapl_;")
one_result = cur.fetchone()
print(one_result)

(7288,)


In [15]:
import pandas as pd

In [17]:
df = pd.read_sql("select * from aapl_", conn)

In [18]:
df['summary'][0]

"Apple Inc's Tony Blevins, a key player in the company's supply chain operations, is leaving the iPhone maker, the company confirmed on Thursday, without providing a reason."

In [21]:
df.head(3)

,category,datetime,headline,id,image,related,source,summary,url
0,company,1664488184,Apple senior exec leaves iPhone maker after re...,116345289,https://static.reuters.com/resources/r/?m=02&d...,AAPL,Reuters,"Apple Inc's Tony Blevins, a key player in the ...",https://finnhub.io/api/news?id=45243364a099251...
1,company,1664482051,Exclusive-Senior Indonesian officials targeted...,116342812,,AAPL,Finnhub,More than a dozen senior Indonesian government...,https://finnhub.io/api/news?id=3f9adcc1fd1572f...
2,company,1664474160,House approves scaled-down bill targeting Big ...,116347651,https://images.mktw.net/im-634166/social,AAPL,MarketWatch,The House on Thursday approved sharply scaled-...,https://finnhub.io/api/news?id=87d5ec6a6ccfa60...


In [25]:
data = df[['headline', 'summary']].copy()
data['dt'] = pd.to_datetime(df['datetime'],unit='s')
data

,headline,summary,dt
0,Apple senior exec leaves iPhone maker after re...,"Apple Inc's Tony Blevins, a key player in the ...",2022-09-29 21:49:44
1,Exclusive-Senior Indonesian officials targeted...,More than a dozen senior Indonesian government...,2022-09-29 20:07:31
2,House approves scaled-down bill targeting Big ...,The House on Thursday approved sharply scaled-...,2022-09-29 17:56:00
3,Tech Down as Apple Slides on Bank of America D...,Shares of technology companies fell sharply as...,2022-09-29 17:22:00
4,"Netflix, YouTube and Google top list of Gen Z'...",YouTube has the highest favorability for Gen Z...,2022-09-29 17:06:00
...,...,...,...
7283,Will iPhone 15 Pro Max Disappoint Fans With Fa...,Looking for stock market analysis and research...,2023-05-29 04:19:00
7284,Stocks Bullish Trend - Elliott Wave Technical ...,Looking for stock market analysis and research...,2023-05-29 04:03:00
7285,Tech-Focused QQQ ETF Crafts New High; Analysts...,Looking for stock market analysis and research...,2023-05-29 02:57:00
7286,A Look At Apple Stock Heading Into The Week,Looking for stock market analysis and research...,2023-05-29 02:33:00


## Пробуем обработать новости

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [55]:
model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", max_length=512, truncation=True, padding='max_length')

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

sentences = ["Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales.",  
             "Bids or offers include at least 1,000 shares and the value of the shares must correspond to at least EUR 4,000.", 
             "Raute reported a loss per share of EUR 0.86 for the first half of 2009 , against EPS of EUR 0.74 in the corresponding period of 2008.", 
             ]
results = nlp(sentences)
print(results)

[{'label': 'positive', 'score': 0.9998133778572083}, {'label': 'neutral', 'score': 0.9997822642326355}, {'label': 'negative', 'score': 0.9877365231513977}]


In [40]:
for i in data['headline'][:10]:
    print(nlp(i)[0]['label'])

neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negative


In [47]:
data['label'].value_counts()

neutral     6134
positive     680
negative     474
Name: label, dtype: int64

In [69]:
%%time

data['len_summary'] = data['summary'].map(lambda x: len(x.split(' ')))

CPU times: user 18.3 ms, sys: 550 µs, total: 18.8 ms
Wall time: 20.8 ms


In [112]:
data.loc[4382,'summary'] #= 'There’s something in the air in Austin, Texas, and it’s not the sound of fans cheering on the Longhorns, the twang of a country guitar, or the smell of Texas BBQ. It’s South by Southwest, an annual celebration that’s billed as both a conference and a festival, or somewhere in the middle, but as organizers and veterans will tell you, ‘you just have to be there.’ It’s a mashup of film, music, tech, and culture. A place where you can watch Ironman lecture a crowd on cyber-security, receive healthcare advice from the Jonas Brothers, and watch a movie premiere in virtual reality, all in one afternoon. But why is this happening? And who’s going? Well, everyone- fans, entrepreneurs, thrill seekers, investors, and they’re all going for the same reason- they’re looking for the next big thing. For the first time in a long time, the tech sector is experiencing a transformation. You’ve read about it in the news, heard about it in the break room, and you’ve seen it with your own eyes on social media. I’m talking about AI and what some are calling “deep tech” – those scientific advancements which include innovations in areas like virtual reality and robotics. Between star studded conversations and out-of-this-world exhibits, SXSW will give us a sneak peak into the future and how these new technologies will change our economy and our world, from the way we work, to the way we play.'

'There’s something in the air in Austin, Texas, and it’s not the sound of fans cheering on the Longhorns, the twang of a country guitar, or the smell of Texas BBQ. It’s South by Southwest, an annual celebration that’s billed as both a conference and a festival, or somewhere in the middle, but as organizers and veterans will tell you, ‘you just have to be there.’ It’s a mashup of film, music, tech, and culture. A place where you can watch Ironman lecture a crowd on cyber-security, receive healthcare advice from the Jonas Brothers, and watch a movie premiere in virtual reality, all in one afternoon. But why is this happening? And who’s going? Well, everyone- fans, entrepreneurs, thrill seekers, investors, and they’re all going for the same reason- they’re looking for the next big thing. For the first time in a long time, the tech sector is experiencing a transformation. You’ve read about it in the news, heard about it in the break room, and you’ve seen it with your own eyes on social med

In [120]:
%%time

data['label_summary'] = data['summary'].map(lambda x: nlp(x)[0]['label'])

CPU times: user 22min 53s, sys: 24.2 s, total: 23min 17s
Wall time: 22min 52s


In [122]:
data['label_summary'].value_counts()

neutral     4424
positive    1782
negative    1082
Name: label_summary, dtype: int64

In [123]:
data.head()

,headline,summary,dt,label,len_summary,label_summary
0,Apple senior exec leaves iPhone maker after re...,"Apple Inc's Tony Blevins, a key player in the ...",2022-09-29 21:49:44,neutral,27,neutral
1,Exclusive-Senior Indonesian officials targeted...,More than a dozen senior Indonesian government...,2022-09-29 20:07:31,neutral,34,neutral
2,House approves scaled-down bill targeting Big ...,The House on Thursday approved sharply scaled-...,2022-09-29 17:56:00,neutral,22,positive
3,Tech Down as Apple Slides on Bank of America D...,Shares of technology companies fell sharply as...,2022-09-29 17:22:00,neutral,25,negative
4,"Netflix, YouTube and Google top list of Gen Z'...",YouTube has the highest favorability for Gen Z...,2022-09-29 17:06:00,neutral,13,neutral


In [124]:
d = {'negative' : -1, 'positive' : 1, 'neutral': 0}
data['label_code'] = data['label_summary'].map(d)

In [125]:
data.head()

,headline,summary,dt,label,len_summary,label_summary,label_code
0,Apple senior exec leaves iPhone maker after re...,"Apple Inc's Tony Blevins, a key player in the ...",2022-09-29 21:49:44,neutral,27,neutral,0
1,Exclusive-Senior Indonesian officials targeted...,More than a dozen senior Indonesian government...,2022-09-29 20:07:31,neutral,34,neutral,0
2,House approves scaled-down bill targeting Big ...,The House on Thursday approved sharply scaled-...,2022-09-29 17:56:00,neutral,22,positive,1
3,Tech Down as Apple Slides on Bank of America D...,Shares of technology companies fell sharply as...,2022-09-29 17:22:00,neutral,25,negative,-1
4,"Netflix, YouTube and Google top list of Gen Z'...",YouTube has the highest favorability for Gen Z...,2022-09-29 17:06:00,neutral,13,neutral,0


In [132]:
new_data = data[data['dt'] >= '2022-09-29']

In [163]:
new_data.tail()

,headline,summary,dt,label,len_summary,label_summary,label_code
7283,Will iPhone 15 Pro Max Disappoint Fans With Fa...,Looking for stock market analysis and research...,2023-05-29 04:19:00,neutral,21,neutral,0
7284,Stocks Bullish Trend - Elliott Wave Technical ...,Looking for stock market analysis and research...,2023-05-29 04:03:00,neutral,21,neutral,0
7285,Tech-Focused QQQ ETF Crafts New High; Analysts...,Looking for stock market analysis and research...,2023-05-29 02:57:00,neutral,21,neutral,0
7286,A Look At Apple Stock Heading Into The Week,Looking for stock market analysis and research...,2023-05-29 02:33:00,neutral,21,neutral,0
7287,3 Top Growth Stocks to Make You the Millionair...,Looking for stock market analysis and research...,2023-05-29 01:52:00,neutral,21,neutral,0


In [135]:
agg_data = new_data.groupby(pd.Grouper(key="dt", freq="d")).sum()

In [181]:
agg_data[agg_data['label_code'] < 0].head()

,len_summary,label_code,sentiment
dt,,,
2022-10-08,256,-1,-1.0
2022-10-09,21,-1,-1.0
2022-10-23,84,-1,-1.0
2022-10-31,1407,-2,-1.0
2022-11-02,1425,-6,-1.0


In [180]:
agg_data['sentiment'] = agg_data['label_code'].map(lambda x: x/abs(x) if x != 0 else 0)

In [182]:
agg_data

,len_summary,label_code,sentiment
dt,,,
2022-09-29,1611,2,1.0
2022-09-30,1319,0,0.0
2022-10-01,369,2,1.0
2022-10-02,344,2,1.0
2022-10-03,1071,7,1.0
...,...,...,...
2023-05-25,1465,9,1.0
2023-05-26,1290,5,1.0
2023-05-27,423,3,1.0


## Финансовые данные

In [152]:
import yfinance as yf

In [164]:
stocks = yf.download(init_.tickers[0], init_.start_date, init_.end_date)

TypeError: can only concatenate str (not "int") to str

In [173]:
stocks['delta'] = stocks['Adj Close'].diff().shift(-1)
stocks

,Open,High,Low,Close,Adj Close,Volume,delta
Date,,,,,,,
2022-09-29,146.100006,146.720001,140.679993,142.479996,141.831009,128138200,-4.260529
2022-09-30,141.279999,143.100006,138.000000,138.199997,137.570480,124925300,4.230652
2022-10-03,138.210007,143.070007,137.690002,142.449997,141.801132,114311700,3.633377
2022-10-04,145.029999,146.220001,144.259995,146.100006,145.434509,87830100,0.298630
2022-10-05,144.070007,147.380005,143.009995,146.399994,145.733139,79471000,-0.965576
...,...,...,...,...,...,...,...
2023-05-22,173.979996,174.710007,173.449997,174.199997,174.199997,43570900,-2.639999
2023-05-23,173.130005,173.380005,171.279999,171.559998,171.559998,50747300,0.279999
2023-05-24,171.089996,172.419998,170.520004,171.839996,171.839996,45143500,1.150009


## Джойн в единую таблицу

In [185]:
result = agg_data.join(stocks, on='dt', how='inner')

In [203]:
dataframe = result[['label_code', 'delta']].copy()
dataframe = dataframe[:-1]
dataframe

,label_code,delta
dt,,
2022-09-29,2,-4.260529
2022-09-30,0,4.230652
2022-10-03,7,3.633377
2022-10-04,15,0.298630
2022-10-05,16,-0.965576
...,...,...
2023-05-19,3,-0.960007
2023-05-22,5,-2.639999
2023-05-23,11,0.279999
